In [22]:
# ---------- Preparations ----------
# Load Libraries
library(Spectra)
library(MsBackendMgf)
library(MsBackendHmdb)
library(MsCoreUtils)
library(MsBackendMsp)
library(readr)
library(dplyr)
# 3 dependencies for latest MassBank version
library(rvest)
library(stringr)
library(xml2)
options(warn=-1)
#library("mzR")

In [23]:
input_dir <- getwd()
input_dir

[1] "/Users/mahnoorzulfiqar/OneDriveUNI/MAW"

In [24]:
#install.packages(c("BiocManager", "remotes"))
#BiocManager::install("RforMassSpectrometry/CompoundDb")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org




In [16]:
#install.packages("curl")


Die heruntergeladenen Binärpakete sind in 
	/var/folders/3j/00yb0rbj5m9b2pn32c3v6d_c0000gn/T//RtmpIu8UOi/downloaded_packages


In [17]:
#' Download the CompDb database using curl
library(curl)
dbname <- "CompDb.Hsapiens.HMDB.5.0.sqlite"
db_file <- file.path(tempdir(), dbname)
curl_download(
    paste0("https://github.com/jorainer/MetaboAnnotationTutorials/",
           "releases/download/2021-11-02/", dbname),
    destfile = db_file)

Using libcurl 7.79.1 with LibreSSL/3.3.5


Attache Paket: ‘curl’


Das folgende Objekt ist maskiert ‘package:readr’:

    parse_date




In [19]:
#' Load a CompDb database with compound annotation from HMDB
cdb <- CompDb(db_file)
cdb

Lade nötiges Paket: AnnotationFilter



class: CompDb 
 data source: HMDB 
 version: 5.0 
 organism: Hsapiens 
 compound count: 217776 
 MS/MS spectra count: 64920 

In [20]:
hmdb <- Spectra(cdb)

In [21]:
hmdb

MSn data (Spectra) with 64920 spectra in a MsBackendCompDb backend:
        msLevel precursorMz  polarity
      <integer>   <numeric> <integer>
1            NA          NA         1
2            NA          NA         1
3            NA          NA         1
4            NA          NA         1
5            NA          NA         1
...         ...         ...       ...
64916        NA          NA         0
64917        NA          NA         0
64918        NA          NA         0
64919        NA          NA         0
64920        NA          NA         1
 ... 32 more variables/columns.
 Use  'spectraVariables' to list all of them.
 data source: HMDB 
 version: 5.0 
 organism: Hsapiens 

In [25]:
download_specDB <- function(input_dir, db = "all"){

    if (dir.exists(input_dir)){
        # Track Time 
        start_time <- Sys.time()

        # only input available as of now
        databases <- 'gnps, hmdb, mbank, all'
        # creat a summary file, open and store timings of download and version if possible
        if (!(file.exists(paste(input_dir, "/summaryFile.txt", sep = "")))){
            summaryFile <- paste(input_dir, "/summaryFile.txt", sep = "")
            file.create(summaryFile, recursive = TRUE)
        }
        else{
            summaryFile <- paste(input_dir, "/summaryFile.txt", sep = "")
        }
        file.conn <- file(summaryFile)
        open(file.conn, open = "at")
        # gnps
        if (db == "all" || db =="gnps"){
            # Download file
            system(paste("wget -P", 
                         input_dir,
                         "https://gnps-external.ucsd.edu/gnpslibrary/ALL_GNPS.mgf",
                         sep =  " "))

            # load the spectra into MsBackendMgf
            gnpsdb <- Spectra(paste(input_dir, "/ALL_GNPS.mgf", sep = ''), source = MsBackendMgf())
            save(gnpsdb, file = paste(input_dir,"/gnps.rda", sep = ""))

            # delete the database in its format to free up space
            system(paste("rm", (paste(input_dir, "/ALL_GNPS.mgf", sep = '')), sep = " "))

            writeLines(paste("GNPS saved at", Sys.time(), sep=" "),con=file.conn)
        }
        #mbank
        if (db == "all" || db =="mbank"){

            #print("MassBank WORKS")

            page <- read_html("https://github.com/MassBank/MassBank-data/releases")
            page %>%
                html_nodes("a") %>%       # find all links
                html_attr("href") %>%     # get the url
                str_subset("MassBank_NIST.msp") -> tmp # find those that have the name MassBank_NIST.msp

            #download file
            system(paste("wget ",
                         "https://github.com", tmp[1], 
                         sep =  ""))

            mbank <- Spectra(paste(input_dir, "/MassBank_NIST.msp", sep = ''), source = MsBackendMsp())
            save(mbank, file = paste(input_dir,"/mbankNIST.rda", sep = ""))

            # delete the database in its format to free up space
            system(paste("rm", (paste(input_dir, "/MassBank_NIST.msp", sep = '')), sep = " "))

            # obtain the month and year for the database release to add to summary
            res <- str_match(tmp[1], "download/\\s*(.*?)\\s*/MassBank_NIST")

            writeLines(paste("MassBank saved at", Sys.time(), "with release version", res[,2], sep=" "),con=file.conn)
        }
        
        #mbank
        if (db == "all" || db =="hmdb"){
            dbname <- "CompDb.Hsapiens.HMDB.5.0.sqlite"
            db_file <- file.path(tempdir(), dbname)
            curl_download(
                paste0("https://github.com/jorainer/MetaboAnnotationTutorials/",
               "releases/download/2021-11-02/", dbname),
                destfile = db_file)
            #' Load a CompDb database with compound annotation from HMDB
            cdb <- CompDb(db_file)
            hmdb_version <- 
            hmdb <- Spectra(cdb)
            writeLines(paste("HMDB saved at", Sys.time(), "with release version", hmdb_version, sep=" "),con=file.conn)
        }
        
        #wrong input error message
        else if (!grepl(db, databases, fixed = TRUE)){
            stop("Wrong db input. Following inputs apply: gnps, hmdb, mbank or all")
        }
        close(file.conn)
        end_time <- Sys.time()
        print(end_time - start_time)
    }
    else{
        stop("Your input_dir is incorrect. Please provide the directory where all your input files are stored.")
    }
}


In [ ]:
download_specDB(input_dir, db = "all")

Start data import from 1 files ... 
done



In [11]:
# extract HMDB Current version
#html <- read_html("https://hmdb.ca/downloads")
#strings <- html%>% html_elements("a") %>% html_text2()
#ls <- unique(strings)
#hmdb_curr_ver <- c()
#for (i in ls){
    #if (grepl("Current", i)){
    #hmdb_curr_ver<- c(i, hmdb_curr_ver)
    #}
#}
#hmdb_curr_ver

[1] "Current Version (5.0)"

In [13]:
####### Download and unzip ######

#Download file predicted MSMS spectra
#system(paste("wget",
             #"https://hmdb.ca/system/downloads/current/spectral_data/spectra_xml/hmdb_predicted_msms_spectra.zip",
            # sep = " "))
# unzip
#system(paste("unzip", "hmdb_predicted_msms_spectra.zip", "-d",  paste(input_dir, "/hmdb_predicted_msms_spectra", sep = ""), sep = " "))


#Download file experimental MSMS spectra
#system(paste("wget",
            # "https://hmdb.ca/system/downloads/current/spectral_data/spectra_xml/hmdb_experimental_msms_spectra.zip",
            # sep = " "))
# unzip
#system(paste("unzip", "hmdb_experimental_msms_spectra.zip", "-d", paste(input_dir, "/hmdb_experimental_msms_spectra", sep = ""), sep = " "))



In [2]:
#path <- "/Users/mahnoorzulfiqar/OneDriveUNI/MZML/hmdb_predicted_msms_spectra"
#fls <- dir(path, pattern = "ms_ms_spectrum", full.names = TRUE)

In [16]:
#fls

In [ ]:
#sps_hmdb <- Spectra(fls, source = MsBackendHmdbXml(), nonStop = TRUE,
                    #backend = MsBackendDataFrame())

Start data import from 1797275 files ... 
